In [ ]:

from datetime import datetime
from langchain_ollama import ChatOllama
from langgraph.prebuilt import create_react_agent

from langgraph.checkpoint.memory import MemorySaver
from langchain_core.tools import tool

# from src.utils.api_helpers import query_arxiv
from typing_extensions import TypedDict, Literal

from langchain_ollama import ChatOllama
from langchain_core.messages import AnyMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.types import Command

model = ChatOllama(model="mistral:7b")

from typing_extensions import TypedDict, Literal

from langchain_core.messages import AnyMessage
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.types import Command

import pprint
from src.utils.api_helpers import get_arxiv_papers


model = ChatOllama(model="mistral:7b")

In [3]:

@tool
def arxiv_tool(keyword: str, num_results: int = 10) -> list[dict]:
    """
    Fetches the latest research papers from ArXiv.
    Covers many topics including computer science, physics, math, etc.
    :param keyword: The keyword to search for.
    """
    return get_arxiv_papers(keyword, num_results)


tools = [arxiv_tool]
model = ChatOllama(model="mistral:7b")

system_prompt = (
    "You are a helpful idea generation agent, but you don't"
    " know much about research, so you use arXiv API to get"
    " the latest research."
    " When generating ideas, don't generate more than 5."
    " When you suggest ideas, share the pros and cons." 
    " Focus on impact of the work, novelty and feasibility."
    " If you're suggesting ideas, make sure to look at current"
    " research, don't just suggest random ideas."
    " IMPORTANT: always use the arXiv API to get the latest research."
    " IMPORTANT: if the user asks for reading, always use the arXiv API."
)

# system_prompt="Use the arxiv papers tool"

graph = create_react_agent(
    model, tools, checkpointer=MemorySaver(), state_modifier=system_prompt) # ,
    # state_modifier=system_prompt)
config = {"configurable": {"thread_id": "thread-1"}}

def print_stream(graph, inputs, config):
    for s in graph.stream(inputs, config, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
    return message

In [7]:
inputs = {"messages": [("user", "What are the research papers on arxiv right now?")]}
message = print_stream(graph, inputs, config)

================================ Human Message =================================

What are the research papers on arxiv right now?
================================== Ai Message ==================================

 To find out the latest research papers on arXiv, I will utilize the arxiv_tool function you provided. Here is a list of 5 papers from various categories:

1. Title: Deep Graph Infomax for Semi-supervised Learning (arXiv:2301.08749)
   - Category: Computer Science
   - Pros: This paper introduces a new method called Deep Graph Infomax for semi-supervised learning, which can potentially improve the performance of AI models in low-data regimes and reduce reliance on large labeled datasets.
   - Cons: The method might not be as effective when dealing with complex and high-dimensional graph structures due to its computational complexity.

2. Title: A Pragmatic Framework for Interactive, Collaborative AI Systems (arXiv:2301.08769)
   - Category: Computer Science
   - Pros: This pap

In [ ]:

# Define a helper for each of the agent nodes to call
def call_llm(messages: list[AnyMessage], target_agent_nodes: list[str]):
    """Call LLM with structured output to get a natural language response as well as a target agent (node) to go to next.

    Args:
        messages: list of messages to pass to the LLM
        target_agents: list of the node names of the target agents to navigate to
    """
    # define JSON schema for the structured output:
    # - model's text response (`response`)
    # - name of the node to go to next (or 'finish')
    # see more on structured output here https://python.langchain.com/docs/concepts/structured_outputs
    json_schema = {
        "name": "Response",
        "parameters": {
            "type": "object",
            "properties": {
                "response": {
                    "type": "string",
                    "description": "A human readable response to the original question. Does not need to be a final response. Will be streamed back to the user.",
                },
                "goto": {
                    "enum": [*target_agent_nodes, "__end__"],
                    "type": "string",
                    "description": "The next agent to call, or __end__ if the user's query has been resolved. Must be one of the specified values.",
                },
            },
            "required": ["response", "goto"],
        },
    }
    response = model.with_structured_output(json_schema).invoke(messages)
    return response


def travel_advisor(
    state: MessagesState,
) -> Command[Literal["sightseeing_advisor", "hotel_advisor", "__end__"]]:
    system_prompt = (
        "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc). "
        "If you need specific sightseeing recommendations, ask 'sightseeing_advisor' for help. "
        "If you need hotel recommendations, ask 'hotel_advisor' for help. "
        "If you have enough information to respond to the user, return 'finish'. "
        "Never mention other agents by name."
    )
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    target_agent_nodes = ["sightseeing_advisor", "hotel_advisor"]
    response = call_llm(messages, target_agent_nodes)
    if response is None:
        return Command(goto="__end__", update={"messages": ["Sorry, LLM not responding!"]})
    ai_msg = {"role": "ai", "content": response["response"], "name": "travel_advisor"}
    # handoff to another agent or halt
    return Command(goto=response["goto"], update={"messages": ai_msg})


def sightseeing_advisor(
    state: MessagesState,
) -> Command[Literal["travel_advisor", "hotel_advisor", "__end__"]]:
    system_prompt = (
        "You are a travel expert that can provide specific sightseeing recommendations for a given destination. "
        "If you need general travel help, go to 'travel_advisor' for help. "
        "If you need hotel recommendations, go to 'hotel_advisor' for help. "
        "If you have enough information to respond to the user, return 'finish'. "
        "Never mention other agents by name."
    )
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    target_agent_nodes = ["travel_advisor", "hotel_advisor"]
    response = call_llm(messages, target_agent_nodes)
    ai_msg = {
        "role": "ai",
        "content": response["response"],
        "name": "sightseeing_advisor",
    }
    # handoff to another agent or halt
    return Command(goto=response["goto"], update={"messages": ai_msg})


def hotel_advisor(
    state: MessagesState,
) -> Command[Literal["travel_advisor", "sightseeing_advisor", "__end__"]]:
    system_prompt = (
        "You are a travel expert that can provide hotel recommendations for a given destination. "
        "If you need general travel help, ask 'travel_advisor' for help. "
        "If you need specific sightseeing recommendations, ask 'sightseeing_advisor' for help. "
        "If you have enough information to respond to the user, return 'finish'. "
        "Never mention other agents by name."
    )
    messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    target_agent_nodes = ["travel_advisor", "sightseeing_advisor"]
    response = call_llm(messages, target_agent_nodes)
    ai_msg = {"role": "ai", "content": response["response"], "name": "hotel_advisor"}
    # handoff to another agent or halt
    return Command(goto=response["goto"], update={"messages": ai_msg})


builder = StateGraph(MessagesState)
builder.add_node("travel_advisor", travel_advisor)
builder.add_node("sightseeing_advisor", sightseeing_advisor)
builder.add_node("hotel_advisor", hotel_advisor)
# we'll always start with a general travel advisor
builder.add_edge(START, "travel_advisor")

graph = builder.compile()

In [ ]:
def call_llm(messages: list[AnyMessage], target_agent_nodes: list[str]):
    """Call LLM with structured output to get a natural language response as well as a target agent (node) to go to next.

    Args:
        messages: list of messages to pass to the LLM
        target_agents: list of the node names of the target agents to navigate to
    """
    # define JSON schema for the structured output:
    # - model's text response (`response`)
    # - name of the node to go to next (or 'finish')
    # see more on structured output here https://python.langchain.com/docs/concepts/structured_outputs
    json_schema = {
        "name": "Response",
        "parameters": {
            "type": "object",
            "properties": {
                "response": {
                    "type": "string",
                    "description": "A human readable response to the original question. Does not need to be a final response. Will be streamed back to the user.",
                },
                "goto": {
                    "enum": [*target_agent_nodes, "__end__"],
                    "type": "string",
                    "description": "The next agent to call, or __end__ if the user's query has been resolved. Must be one of the specified values.",
                },
                "interest": {
                    "type": "string",
                    "description": "A user interest if it can be extracted.",
                },
                "interests": {
                    "type": "list",
                    "description": "A list of user interests if it can be extracted.",
                    "items": {
                        "type": "string",
                        "descritption": "A user interest, no more than a few words.",
                    },
                }
            },
        },
    }
    response = model.with_structured_output(json_schema).invoke(messages)
    return response

def generator(state):
    system_prompt = (
        "You are a research idea generation agent. You can generate research ideas based on user interests. "
        "If you need more information about the user's interests, ask 'research'. "
        "If you have enough information to respond to the user, return 'finish'. "
        "Never mention other agents by name."
    )
    # messages = [{"role": "system", "content": system_prompt}] + state["messages"]
    system_message = HumanMessage(content=system_prompt)
    system_message = ("user", system_prompt)
    messages = (system_message,) + state["messages"]
    print(messages)
    # messages = (system_message,)# state["messages"]
    target_agent_nodes = ["research"]
    response = None
    i = 9
    while response is None and i < 10:
        response = call_llm(messages, target_agent_nodes)
        print(response) if response is None else 1
        i += 1
    if response is None:
        return Command(goto="__end__", update={"messages": ["Sorry, LLM not responding!"]})
    ai_msg = {"role": "ai", "content": response["response"], "name": "generator"}
    # handoff to another agent or halt
    return Command(goto=response["goto"], update={"messages": ai_msg})


from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

tm = HumanMessage(content="I'm interested in GANs")
tm = ("user", "I'm interested in GANs")
res = generator({"messages": (tm,)})

import pprint

pprint.pprint(res)

(('user', "You are a research idea generation agent. You can generate research ideas based on user interests. If you need more information about the user's interests, ask 'research'. If you have enough information to respond to the user, return 'finish'. Never mention other agents by name."), ('user', "I'm interested in GANs"))
None
Command(update={'messages': ['Sorry, LLM not responding!']}, goto='__end__')


In [ ]:
from typing import Optional
from pydantic import BaseModel, Field

from src.utils.api_helpers import build_api_tools


class IdeasList(BaseModel):
    """List of research ideas."""

    ideas: list[str] = Field(description="A list of research ideas")
    # user_response: str = Field(description="Text response to share ideas with user")
    goto: Optional[str] = Field(description="The next agent to call ('research', 'feedback'), or __end__ if the user's query has been resolved. Must be one of the specified values.")

llm = ChatOllama(model="mistral:7b")
structured_llm = llm.with_structured_output(IdeasList)


tools = build_api_tools()

agent = create_react_agent(llm, tools=tools)

res = agent.invoke({"messages": [("user", "What are the latest research papers on arXiv?")]})

pprint.pprint(res)
# model.invoke([("user", system_prompt),("user", "hi, hello, tell me about GANs")])

AttributeError: 'RunnableSequence' object has no attribute 'bind_tools'

In [63]:
pprint.pprint(res.ideas)
pprint.pprint(res.goto)

['1. Investigating the application of Generative Adversarial Networks (GANs) '
 'in simulating images and videos of black holes.',
 '2. Developing a model to predict the formation and evolution of black holes '
 'using GANs.',
 '3. Studying the impact of black hole environments on the properties and '
 'behavior of accretion disks using GANs.',
 '4. Exploring the use of GANs in generating realistic light curves and '
 'spectra for various types of black holes.',
 '5. Investigating the potential role of black holes in the cosmic web '
 'structure using GANs.']
'__end__'


In [42]:

system_prompt = (
    "You are a research idea generation agent. You can generate research ideas based on user interests. "
    "If you need more information about the user's interests, ask 'research'. "
    "If you have enough information to respond to the user, return 'finish'. "
    "Never mention other agents by name."
)
# system_prompt = "hi"

tm = ("user", "What are the latest papers in GANs?")
# tm = ("user", "I'm interested in GANs")
tm = (("user", "hello"))
tm = (("user", system_prompt), ("user", "hello"))
# tm = (('user', "You are a research idea generation agent. You can generate research ideas based on user interests. If you need more information about the user's interests, ask 'research'. If you have enough information to respond to the user, return 'finish'. Never mention other agents by name."), ('user', "I'm interested in GANs"))

# returns None randomly
pprint.pprint(call_llm(tm, target_agent_nodes=[]))

{'goto': 'generate_research_ideas'}


In [115]:
system_prompt = (
    "You are a research idea generation agent. You can generate research ideas based on user interests. "
    "If you need more information about the user's interests, ask 'research'. "
    "If you have enough information to respond to the user, return 'finish'. "
    "Never mention other agents by name."
)
# system_prompt = "hi"

tm = {"role": "user", "content": "What are the latest papers in GANs?"}
# tm = ("user", "I'm interested in GANs")
hi = {"role": "user", "content": "hello"}
tm = [hi, hi, tm]
# tm = (("user", system_prompt), ("user", "hello"))
# tm = [{"role": "user", "content": system_prompt}, tm]
# tm = (('user', "You are a research idea generation agent. You can generate research ideas based on user interests. If you need more information about the user's interests, ask 'research'. If you have enough information to respond to the user, return 'finish'. Never mention other agents by name."), ('user', "I'm interested in GANs"))
print(tm)

# returns None randomly
pprint.pprint(call_llm(tm, target_agent_nodes=[]))

[{'role': 'user', 'content': 'hello'}, {'role': 'user', 'content': 'hello'}, {'role': 'user', 'content': 'What are the latest papers in GANs?'}]
None


In [10]:
import requests 
from dotenv import load_dotenv
import os

load_dotenv()

def get_response_cgu(messages, model="llama3:latest"):
    """
    Calls the CGU API to get a response for the given messages.
    """
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {os.getenv("CGU_API_KEY")}',
    }

    models = ["taide:latest", "llama3:latest"]

    json_data = {
        'model': models[1],
        'stream': 'false',
        'messages': messages,
    }

    response = requests.post('http://120.126.23.245:32264/ollama/api/chat', headers=headers, json=json_data)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error calling CGU API")
        return None

get_response_cgu([{"role": "user", "content": "Convert the following to Chinese: hello"}], model="taide:latest")

{'model': 'llama3:latest',
 'created_at': '2024-12-18T06:47:32.001008383Z',
 'message': {'role': 'assistant',
  'content': 'The translation of "hello" in Chinese is:\n\n\n\n(nǐ hǎo)\n\nNote:\n\n* (nǐ) means "you"\n* (hǎo) means "good" or "well", but when used as a greeting, it\'s more like "hello" or "hi".\n\nSo, (nǐ hǎo) literally means "You good", but it\'s commonly used as a friendly greeting.'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 5152418265,
 'load_duration': 4097013410,
 'prompt_eval_count': 17,
 'prompt_eval_duration': 33108000,
 'eval_count': 85,
 'eval_duration': 1020348000}

In [ ]:
def check_weather(location: str, at_time: datetime | None = None) -> str:
    """Return the weather forecast for the specified location."""
    return f"It's always sunny in {location}"


tools = [check_weather]
model = ChatOllama(model="mistral")


def test_simple_react_agent():
    system_prompt = (
        "You are a helpful bot named Fred. Call tools yourself, don't return code to the user."
    )
    graph = create_react_agent(model, tools, state_modifier=system_prompt)
    inputs = {"messages": [("user", "What's your name? And what's the weather in SF?")]}

    tool_results = []

    for s in graph.stream(inputs, stream_mode="values"):
        message = s["messages"][-1]
        if isinstance(message, ToolMessage):
            tool_results.append(message)
        """if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()"""

    return tool_results

state = test_simple_react_agent()

In [29]:
from langchain_core.messages import ToolMessage

[s.content for s in state["messages"] if isinstance(s, ToolMessage)]

["It's always sunny in San Francisco"]